In [8]:
import xml.etree.ElementTree as ET
import pandas as pd


def xml_to_csv(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    
    data = []
    paragrafnummer = 1  # Initialize XML structure
    
    for paragraph in root.findall('.//paragraph'):
        for sentence in paragraph.findall('.//sentence'):
            for word in sentence.findall('.//w'):
                data.append({
                    "paragrafnummer": paragrafnummer,
                    "token": word.text,
                    "msd": word.get('msd', ''),
                    "lemma": word.get('lemma', ''),
                    "lex": word.get('lex', ''),
                    "sense": word.get('sense', ''),
                    "complemgram": word.get('complemgram', ''),
                    "compwf": word.get('compwf', ''),
                    "sentimentclass": word.get('sentimentclass', ''),
                    "deprel": word.get('deprel', '')
                })
        paragrafnummer += 1  # Increment for each paragraph
    
    df = pd.DataFrame(data)
    csv_path = xml_path.replace('.xml', '.csv')  # Or specify your own path
    df.to_csv(csv_path, index=False)
    return csv_path

# Use the function
xml_path = 'testtestdata.xml'
csv_path = xml_to_csv(xml_path)
print(f"CSV file created at: {csv_path}")


CSV file created at: testtestdata.csv


In [9]:
testtestdata_df= pd.read_csv('testtestdata.csv', encoding='utf-8')
testtestorg_df=pd.read_csv('testtestorg.csv', header=None, names=['paragrafnummer', 'sentence', 'category'], encoding='utf-8')

In [10]:


# Add paragrafnummer from testtestorg to testtestdata
testtestdata_df['sentimentval'] = testtestdata_df['paragrafnummer'].apply(
    lambda x: testtestorg_df.at[x-1, 'paragrafnummer'] if x-1 < len(testtestorg_df) else None
)

# Map the category from testtestorg to testtestdata
testtestdata_df['category'] = testtestdata_df['paragrafnummer'].apply(
    lambda x: testtestorg_df.at[x-1, 'category'] if x-1 < len(testtestorg_df) else None
)

# Replace âˆ’ with - in the entire DataFrame
testtestdata_df = testtestdata_df.replace('−', '-', regex=True)

# Save the final corrected DataFrame to a new CSV file
final_csv_path = 'final_corrected_testtestdata.csv'
testtestdata_df.to_csv(final_csv_path, index=False)

print(f"Final corrected CSV saved to: {final_csv_path}")


Final corrected CSV saved to: final_corrected_testtestdata.csv


In [11]:
df= pd.read_csv('final_corrected_testtestdata.csv')
unique_values = df['sentimentval'].unique()
print(unique_values)


[ 1  0 -1]


In [17]:
# Read the CSV file into a DataFrame
predictDf = pd.read_csv('final_corrected_testtestdata.csv')
predictDf = predictDf.dropna()

# Convert sentimentclass values to numerical values
sentiment_mapping = {'positive': 1, 'neutral': 0, 'negative': -1}
predictDf['sentimentclass'] = predictDf['sentimentclass'].map(sentiment_mapping)

# Group by paragrafnummer and calculate the average sentimentclass
average_sentiment = predictDf.groupby('paragrafnummer')['sentimentclass'].mean()



predictDf['average_sentiment'] = average_sentiment
predictDf

,paragrafnummer,token,msd,lemma,lex,sense,complemgram,compwf,sentimentclass,deprel,sentimentval,category,average_sentiment
1,1,blev,VB.PRT.AKT,|bli|,|bli..vb.1|,|bli..1:0.866|bli..2:0.134|,|,|,0,ROOT,1,egna meningar,0.5
5,1,stöd,NN.NEU.SIN.IND.NOM,|stöd|,|stöd..nn.1|,|stöd..1:0.821|stöd..2:0.179|,|,|,1,PA,1,egna meningar,0.0
7,1,fick,VB.PRT.AKT,|få|,|få..vb.1|,|få..1:0.942|få..3:0.058|,|,|,1,ET,1,egna meningar,0.0
10,1,kollegor,NN.UTR.PLU.IND.NOM,|kollega|,|kollega..nn.1|,|kollega..1:-1.000|,|koll..nn.1+lega..nn.1:3.860e-13|koll..nn.2+le...,|koll+legor|kol+legor|,0,PA,1,egna meningar,0.2
13,2,nya,JJ.POS.UTR+NEU.SIN.DEF.NOM,|ny|,|ny..av.1|,|ny..1:-1.000|,|,|,0,AT,1,egna meningar,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1590,169,lägga,VB.INF.AKT,|lägga|,|lägga..vb.1|,|lägga..1:0.972|lägga..2:0.020|lägga..3:0.008|,|,|,0,VG,0,8 Sidor,NaN
1593,169,miljarder,NN.UTR.PLU.IND.NOM,|miljard|,|miljard..nn.1|,|miljard..1:-1.000|,|,|,0,DT,0,8 Sidor,NaN
1594,169,kronor,NN.UTR.PLU.IND.NOM,|krona|,|krona..nn.1|,|krona..1:0.908|krona..3:0.057|krona..2:0.035|,|,|,0,OO,0,8 Sidor,NaN
1596,169,vägar,NN.UTR.PLU.IND.NOM,|väg|,|väg..nn.1|,|väg..2:0.538|väg..1:0.462|,|,|,0,CJ,0,8 Sidor,NaN


In [18]:
# Read the CSV file into a DataFrame
predictDf = pd.read_csv('final_corrected_testtestdata.csv')
predictDf = predictDf.dropna()

# Convert sentimentclass values to numerical values
sentiment_mapping = {'positive': 1, 'neutral': 0, 'negative': -1}
predictDf['sentimentclass'] = predictDf['sentimentclass'].map(sentiment_mapping)

# Group by paragrafnummer and calculate the average sentimentclass
average_sentiment = predictDf.groupby('paragrafnummer')['sentimentclass'].transform('mean')

# Assign the average sentiment to each row
predictDf['average_sentiment'] = average_sentiment

# Fill NaN values in case there are paragraphs without sentimentclass
predictDf['average_sentiment'] = predictDf['average_sentiment'].fillna(method='ffill')

predictDf

C:\Users\ernil\AppData\Local\Temp\ipykernel_22908\1862699497.py:16: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  predictDf['average_sentiment'] = predictDf['average_sentiment'].fillna(method='ffill')


,paragrafnummer,token,msd,lemma,lex,sense,complemgram,compwf,sentimentclass,deprel,sentimentval,category,average_sentiment
1,1,blev,VB.PRT.AKT,|bli|,|bli..vb.1|,|bli..1:0.866|bli..2:0.134|,|,|,0,ROOT,1,egna meningar,0.500000
5,1,stöd,NN.NEU.SIN.IND.NOM,|stöd|,|stöd..nn.1|,|stöd..1:0.821|stöd..2:0.179|,|,|,1,PA,1,egna meningar,0.500000
7,1,fick,VB.PRT.AKT,|få|,|få..vb.1|,|få..1:0.942|få..3:0.058|,|,|,1,ET,1,egna meningar,0.500000
10,1,kollegor,NN.UTR.PLU.IND.NOM,|kollega|,|kollega..nn.1|,|kollega..1:-1.000|,|koll..nn.1+lega..nn.1:3.860e-13|koll..nn.2+le...,|koll+legor|kol+legor|,0,PA,1,egna meningar,0.500000
13,2,nya,JJ.POS.UTR+NEU.SIN.DEF.NOM,|ny|,|ny..av.1|,|ny..1:-1.000|,|,|,0,AT,1,egna meningar,-0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1590,169,lägga,VB.INF.AKT,|lägga|,|lägga..vb.1|,|lägga..1:0.972|lägga..2:0.020|lägga..3:0.008|,|,|,0,VG,0,8 Sidor,0.000000
1593,169,miljarder,NN.UTR.PLU.IND.NOM,|miljard|,|miljard..nn.1|,|miljard..1:-1.000|,|,|,0,DT,0,8 Sidor,0.000000
1594,169,kronor,NN.UTR.PLU.IND.NOM,|krona|,|krona..nn.1|,|krona..1:0.908|krona..3:0.057|krona..2:0.035|,|,|,0,OO,0,8 Sidor,0.000000
1596,169,vägar,NN.UTR.PLU.IND.NOM,|väg|,|väg..nn.1|,|väg..2:0.538|väg..1:0.462|,|,|,0,CJ,0,8 Sidor,0.000000
